<a href="https://colab.research.google.com/github/Pavlos01232/Match_Outcome_Prediction/blob/main/MO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Introduction:
=====================
In this project we dive into the world of sports analytics. Specifally football matches. Our aim is to predict how many goals Team A scores against Team B utilizing data and preivious match statistics for mutliple teams.

We will use neural networks and machine learning techniques to analyze each team's historical match data. Some hisotical match data that we are going to be using is are shots on target, red cards, goals etc. Using these techniques we can make informed predicitons about future matches and try to predicit the final score between 2 teams.

Plan to test the model:
======================

Eventually need to train this on some data frames and then
calculate the cross entropy loss on an *unseen* frame.
This might involve putting in a feature to save the weight arrays a,b,c to a file.
To read the cross entropy loss on an unseen data set after
reading them in,  this very program would work,  with
eta=0 and tau=0.

As it works now:
================

Training with eta = tau = 0   -- or just looking at the cross-entropy
loss before trainig will give minus the log of the
the max likelihood of the bookies' estimates



Training with eta not zero but tau =0 and looking at the cross-entropy
loss will give minus the log of the max likelihood for the 1980's models



Training with eta not zero and tau a (much smaller) nonzero number, now
the cross-entropy loss will show how much an extra neural stage on the
back can improve on ordinary max likelihood.



Historic Bookies Estimate:
==========================

The historic bookies estimate for goals landed by team i against team j is

a_i b_j

where

a_i = A_i C^{-1/2}
b_j = B_j C^{-1/2}

A_i = goals landed per game by team i
B_j = goals conceded per game by team j
C   = average goals per game of all teams


The historic 1980's max likelihood models
=========================================

Starting with Maher, the 1980's max likelihood model starts with this guess and
perfects it by max likelihood, when team i lands k goals agaist team j
loss was minus the log of the predicted probability by Poisson

- log ( e^{-a_ib_j}(a_ib_j)^k /k!)


A gitgub user said chi squared shows HST,AST,HR,AR have a significant effect

HST = home shots on target
AST = away shots on target
HR  = home red cards
AR  = away red cards


New Cross-entropy AI model with a neural layer
==========================================

Since gradient descent generalizes max likelihood we can replace a_ib_j by

a_i b_j  +  c_0 sigma ( c_1 HST + c_2 HR ) b_j

when i is the home team and

a_i b_j  +  c_0 sigma ( c_1 AST + c_2 AR ) b_j

when i is the away team, with sigma being the sigmoid function


   sigma(x)=softmax(0,x) = e^x/(e^0+e^x) = 1/(1=e^{-x})



This puts a *neural layer* behind the standard max likelihood model from the 1980s

The weights are now the a_i,  b_i,   and c_i

Since the c_i are shared by all teams the training rate for the c_i should be lower

As in the model which this generalizes, the training is cross entropy versus the Poisson distribution.


A comment about the way the databases are stored, variables like HST and AST refer to 'home'
and 'away' team but we do not make a distinction between home versus away.

This means that each row of a data table is interpreted as if it were two rows,
one giving information about team i against team j, the other giving information
about team j against team i.

For instance to calculate the average goals scored by any team over all games
each row gives goals scored by a home team and goals scored by an away team
and we have to add 2 to total games.

That is when we say total games it really means the sum over all teams
of the number of games that team played in, which is twice the number
of games.

That explains the line  totalgames=totalgames+2 each time a row is read in.

There is no need to change this architecture to include things causing a home
team advantage. This starts with a constant taking the value 1 in the first line

loss -=  ...

and taking the value 0 in the second line

loss -= ...

First we start of our code by importing any relavant packages and defining the sigmoid function the takes tensor(x) input and returns a tensor value.


In [3]:
import torch as t
import torch.nn as n
import pandas as pd
import os
import math


def sigma(x):
  return t.exp(x)/(t.exp(t.tensor(0.0))+t.exp(x))

t.set_printoptions(precision=10)

first = "https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/PL"
last = ".csv?raw=true"
x = t.tensor(3)



tensor(0.9525741339)


Data
=======================
First we need some data to work with, due to our approach we need some data from each teams match history. We will be using this https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/PL0304.csv?raw=true.

This table shows football match results and related statistics to do with the game. First lets talk about they keys to the data:
*   Div = League Division
*   Date = Match Date (dd/mm/yy)
*   Time = Time of match kick off
*   HomeTeam = Home Team
*   AwayTeam = Away Team
*   FTHG and HG = Full Time Home Team Goals
*   FTAG and AG = Full Time Away Team Goals
*   FTR and Res = Full Time Result (H=Home Win, D=Draw, A=Away Win)
*   HTHG = Half Time Home Team Goals
*   HTAG = Half Time Away Team Goals
*   HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)

The table also shows match statistics where its available as some data might be missing however the scale of the data it more likely to not affect the bigger pitcutre or the predicition model.

Note that Free Kicks Conceeded includes fouls, offsides and any other offense commmitted and will always be equal to or higher than the number of fouls. Fouls make up the vast majority of Free Kicks Conceded. Free Kicks Conceded are shown when specific data on Fouls are not available (France 2nd, Belgium 1st and Greece 1st divisions).

Note also that English and Scottish yellow cards do not include the initial yellow card when a second is shown to a player converting it into a red, but this is included as a yellow (plus red) for European games.


*   Attendance = Crowd Attendance
*   Referee = Match Referee
*   HS = Home Team Shots
*   HST = Home Team Shots on Target
*   AST = Away Team Shots on Target
*   HHW = Home Team Hit Woodwork
*   AHW = Away Team Hit Woodwork
*   HC = Home Team Corners
*   AC = Away Team Corners
*   HF = Home Team Fouls Committed
*   AF = Away Team Fouls Committed
*   HFKC = Home Team Free Kicks Conceded
*   AFKC = Away Team Free Kicks Conceded
*   HO = Home Team Offsides
*   AO = Away Team Offsides
*   HY = Home Team Yellow Cards
*   AY = Away Team Yellow Cards
*   HR = Home Team Red Cards
*   AR = Away Team Red Cards
*   HBP = Home Team Bookings Points (10 = yellow, 25 = red)
*   ABP = Away Team Bookings Points (10 = yellow, 25 = red)


The table also includes some betting odds keys that we wont dive into.































Data Frame Column Selection and Combination Utility
=================================
In this cell we create a function called combine that takes inputs dataFrames and columnNames. The function extracts columns from each data frame and combines it into a list. The reason for this is to create a consolidated dataset so its easier to extract and work with.


In [ ]:

def combine(dataFrames,columnNames):
 t=[]
 for i in range(0,len(dataFrames)):
    theseColumns=dataFrames[i].columns.values[0].split(",")
    for j in range(0 ,len(dataFrames[i])):
      row=dataFrames[i].values[j][0].split(",")
      newEntry=[]
      for k in range(0, len(columnNames)):
         for m in range(0, len(theseColumns)):
             if(columnNames[k]==theseColumns[m] and m<=len(row)):
                newEntry.append(row[m])
      t.append(newEntry)
 return t





can read years 3 to 23, 13, there's something wrong with 14
since the first two files in the training data are formatted incorrectly
converts csv to dataframe

This code reads CSV files defined from the URL defined above and converts into pandas dataframes.

Something wrong with the data?

In [ ]:
df=[] #Data frame

for i in range(3, 6):
  result = first + str('{:02.0f}'.format(i)) + str('{:02.0f}'.format(i+1)) + last
  x = pd.read_csv(result, sep='\t', encoding = 'unicode_escape', lineterminator='\r')
  df.append(x)




Get an array with each (home) team listed once
from an array of data frames with column "HomeTeam"

In [ ]:
def getTeams(df):
 teams=[]
 homeTeams=combine(df,["HomeTeam"])
 for i in range(len(homeTeams)):
  if(len(homeTeams[i])>0):
   found=False
   for j in range(len(teams)):
    if homeTeams[i][0] == teams[j]:
      found=True
      break
   if found:
    continue
   teams.append(homeTeams[i][0])
 return teams
